In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, PowerTransformer
from keras import models
from keras import layers
import tensorflow as tf
from Informer import Informer
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def create_sequences(X, y, date, window_size):
    seq_X = []
    seq_y = []
    seq_date = []
    for i in range(len(X) - window_size):
        seq_X.append(X[i:i+window_size])
        seq_date.append(date[i:i+window_size])
        seq_y.append(y[i+window_size])

    return np.array(seq_X), np.array(seq_y), np.array(seq_date)

In [ ]:
# read dataset
df= pd.read_csv('Data/Tetuan City power consumption.csv')

In [ ]:
# create date dataset
df_date= pd.DataFrame()
df['DateTime']= pd.to_datetime(df['DateTime'])
df_date['minute']= df['DateTime'].dt.minute
df_date['hour']= df['DateTime'].dt.hour
df_date['weekday']= df['DateTime'].dt.weekday
df_date['month']= df['DateTime'].dt.month

In [ ]:
# create and prepare data & target 
df_data= df.drop(['DateTime', 'Zone 1 Power Consumption'], axis=1)
df_target= df[['Zone 1 Power Consumption']]

# power scaler
Power_scaler= PowerTransformer(method='box-cox')
df_data['normed_general diffuse flows'] = Power_scaler.fit_transform(df_data[['general diffuse flows']])
df_data['normed_diffuse flows'] = Power_scaler.fit_transform(df_data[['diffuse flows']])
df_data['normed_humidity'] = np.clip(df_data['Humidity'], a_min=40, a_max=90)
df_data = df_data.drop(['Humidity', 'general diffuse flows', 'diffuse flows'], axis=1)

# MinMax scaler
MinMax_scaler= MinMaxScaler(feature_range=(0, 1))
df_data= MinMax_scaler.fit_transform(df_data)
df_target= MinMax_scaler.fit_transform(df_target)

# sequencing data
data, target, date= create_sequences(df_data, df_target, df_date.values, window_size=144)

In [ ]:
# prepare date
date_df = pd.DataFrame(date.reshape(-1, date.shape[-1]),
                       columns=['minute', 'hour', 'weekday', 'month'])


date_used = date_df[['hour', 'weekday', 'month']].values
date_used = date_used.reshape(date.shape[0], date.shape[1], 3)

dataset = tf.data.Dataset.from_tensor_slices(((data, date_used), target))

# train/val/test split
train_size = int(0.7 * len(data))
val_size = int(0.15 * len(data))

train_ds = dataset.take(train_size).batch(32).shuffle(100)
val_ds = dataset.skip(train_size).take(val_size).batch(32)
test_ds = dataset.skip(train_size + val_size).batch(32)

In [ ]:
# تعریف مدل
model = Informer(d_model=64, num_heads=4)  # مقادیر رو بسته به منابع تغییر بده
model.compile(optimizer='adam', loss='mse')

# آموزش
history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=10)

# ارزیابی
model.evaluate(test_ds)